<a href="https://colab.research.google.com/github/shreshthi-kumar/skumar-IB2AD-Datascience-GenAI-Submissions/blob/main/Assignment_4/6_02_DNN_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?export=view&id=1xqQczl0FG-qtNA2_WQYuWePW9oU8irqJ)

# 6.02 Dense Neural Network (with PyTorch)
This will expand on our logistic regression example and take us through building our first neural network. If you haven't already, be sure to check (and if neccessary) switch to GPU processing by clicking Runtime > Change runtime type and selecting GPU. We can test this has worked with the following code:

In [1]:
import torch

# Check for GPU availability
print("Num GPUs Available: ", torch.cuda.device_count())

Num GPUs Available:  1


Hopefully your code shows you have 1 GPU available! Next let's get some data. We'll start with another in-built dataset:

In [2]:
# upload an in-built Python (OK semi-in-built) dataset
from sklearn.datasets import load_diabetes

import pandas as pd
import numpy as np

# import the data
data = load_diabetes()
data

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990749, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06833155, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286131, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04688253,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452873, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00422151,  0.00306441]]),
 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59

We are working on a regression problem, with "structured" data which has already been cleaned and normalised. We can skip the usual cleaning/engineering steps. However, we do need to get the data into PyTorch:

In [3]:
# Convert data to PyTorch tensors
X = torch.tensor(data.data, dtype=torch.float32)
y = torch.tensor(data.target, dtype=torch.float32).reshape(-1, 1) # Reshape y to be a column vector

Now our data is stored in tensors we can do train/test splitting as before (in fact we can use sklearn as before):

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([353, 10]) torch.Size([353, 1])
torch.Size([89, 10]) torch.Size([89, 1])


Now we can set up our batches for training. As we have a nice round 400 let's go with batches of 50 (8 batches in total). We'll also seperate the features and labels:

In [5]:
from torch.utils.data import TensorDataset, DataLoader

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)

Now its time to build our model. We'll keep it simple ... a model with an input layer of 10 features and then 2x _Dense_ (fully connected) layers each with 5 neurons and ReLU activation. Our output layer will be size=1 given this is a regression problem and we want a single value output per prediction.

This will be easier to understand if you have read through the logistic regression tutorial.

In [6]:
import torch
import torch.nn as nn

# Define the model
class DiabetesModel(nn.Module):
    def __init__(self):
        super(DiabetesModel, self).__init__()
        # we'll set up the layers as a sequence using nn.Sequential
        self.layers = nn.Sequential(

            # first layer will be a linear layer that has 5x neurons
            # (5x sets of linear regression)
            # the layer takes the 10 features as input (i.e. 10, 5)
            nn.Linear(10, 5),

            nn.ReLU(), # ReLU activation

            # second linear layer again has 5 neurons
            # this time taking the input as the output of the last layer
            # (which had 5x neurons)
            nn.Linear(5, 5),

            nn.ReLU(), # ReLU again

            # last linear layer takes the output from the previous 5 neurons
            # this time its a single output with no activation
            # i.e. this is the predicitons (regression)
            nn.Linear(5, 1)
        )

    def forward(self, x):
        return self.layers(x) # pass the data through the layers

As before we need to create a model object, specify the loss (criterion) and an optimiser (which we cover next week):

In [7]:
import torch.optim as optim

# Initialize the model, loss function, and optimizer
model = DiabetesModel()
criterion = nn.MSELoss() # MSE loss function
optimiser = optim.Adam(model.parameters(), lr=0.001)

Now we can train the model. Again, the logistic regression tutorial (6.01) may help you undertstand this:

In [8]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training loop (example - you'll likely want to add more epochs)
epochs = 100 # 100 epochs

for epoch in range(epochs):
  # use the train_loader to pass the inputs (x) and targets (y)
  for inputs, targets in train_loader:
    # pass to the GPU (hopefully)
    inputs, targets = inputs.to(device), targets.to(device)

    # pass model to GPU as well
    model.to(device)

    model.train() # put the model object in train mode
    optimiser.zero_grad() # reset the gradiants
    outputs = model(inputs) # create outputs
    loss = criterion(outputs, targets) # compare with Y to get loss
    loss.backward() # backpropogate the loss (next week)
    optimiser.step() # # update the parameters based on this round of training

  # every 10 steps we will print out the current loss
    if (epoch+1) % 10 == 0: # modular arithmetic
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {round(loss.item(), 4)}')

Epoch [10/100], Loss: 29865.5234
Epoch [10/100], Loss: 30534.9785
Epoch [10/100], Loss: 29285.166
Epoch [10/100], Loss: 32230.8086
Epoch [10/100], Loss: 24532.2168
Epoch [10/100], Loss: 29031.209
Epoch [10/100], Loss: 33586.3594
Epoch [10/100], Loss: 22287.2656
Epoch [20/100], Loss: 27735.5918
Epoch [20/100], Loss: 31616.2441
Epoch [20/100], Loss: 25491.8848
Epoch [20/100], Loss: 26671.6895
Epoch [20/100], Loss: 36991.2852
Epoch [20/100], Loss: 26737.0098
Epoch [20/100], Loss: 30599.125
Epoch [20/100], Loss: 69860.3594
Epoch [30/100], Loss: 30125.5
Epoch [30/100], Loss: 30625.5293
Epoch [30/100], Loss: 32832.3164
Epoch [30/100], Loss: 31365.5098
Epoch [30/100], Loss: 27192.3262
Epoch [30/100], Loss: 24623.459
Epoch [30/100], Loss: 31313.8418
Epoch [30/100], Loss: 28991.4746
Epoch [40/100], Loss: 30764.0684
Epoch [40/100], Loss: 34924.0859
Epoch [40/100], Loss: 27139.2598
Epoch [40/100], Loss: 30003.3242
Epoch [40/100], Loss: 31167.3398
Epoch [40/100], Loss: 29731.666
Epoch [40/100], Lo

We can see loss is significantly lower at the end than it was at the start. However, it is also bouncing around a little still which suggests the model needs more training (100 epochs is not a lot in deep learning terms). However, let's evaluate as before:

In [9]:
# Evaluation (example)
model.eval() # testing mode
mse_values = [] # collect the MSE scores

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs) # predict the test data

        # Calculate Mean Squared Error
        mse = criterion(outputs, targets) # calcualte mse for the batch
        mse_values.append(mse.item()) # add to the list of MSE values

# Calculate and print the average MSE
avg_mse = np.mean(mse_values)
print(f"Average MSE on test set: {avg_mse}")

Average MSE on test set: 26029.28515625


MSE looks expected given training (no obvious sign of overfitting). However, we probably can get better results with tuning and more epochs.

Let's run the loop again a little differently to collect the predicted values (y_hat) and actuals (y) and add them to a dataset for comparions:

In [10]:
# Evaluation
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(targets.cpu().numpy())

# Create DataFrame
results_df = pd.DataFrame({'Predicted': np.array(predictions).flatten(), 'Actual': np.array(actuals).flatten()})
results_df

,Predicted,Actual
0,0.534942,219.0
1,0.534942,70.0
2,0.534942,202.0
3,0.534942,230.0
4,0.534942,111.0
...,...,...
84,0.534942,153.0
85,0.534942,98.0
86,0.534942,37.0
87,0.534509,63.0


Side-by-side, they don't look great. Can you improve them?

<br><br>

## EXERCISE #1
Try increasing the number of epochs to 1,000 (when the model is fairly well trained then the results printed for each 10x epochs will be fairly stable and not change much). Does this give better results?

<br><br>

## EXERCISE #2 (optional)
Try experimenting with the architecture (number of neurons and/or number of layers). Can we reach an optimal architecture?

In [11]:
# EXERCISE 1: Increase Training to 1000 Epochs

# 1. Re-initialize the model
model_1000 = DiabetesModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_1000.parameters(), lr=0.001)

# 2. Training loop with 1000 epochs
epochs = 1000

print("Starting Training (1000 Epochs)")
for epoch in range(epochs):
    epoch_loss = 0
    # Iterate through batches
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model_1000(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Avg Loss: {round(epoch_loss / len(train_loader), 2)}')

# 3. Evaluation on Test Data
model_1000.eval()
mse_values = []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model_1000(inputs)
        mse_values.append(criterion(outputs, targets).item())

avg_mse_1000 = np.mean(mse_values)
print(f"\nAverage MSE on test set (1000 Epochs): {round(avg_mse_1000, 2)}")

Starting Training (1000 Epochs)
Epoch [100/1000], Avg Loss: 26797.34
Epoch [200/1000], Avg Loss: 8494.76
Epoch [300/1000], Avg Loss: 4414.12
Epoch [400/1000], Avg Loss: 4569.71
Epoch [500/1000], Avg Loss: 3704.9
Epoch [600/1000], Avg Loss: 3623.18
Epoch [700/1000], Avg Loss: 3164.58
Epoch [800/1000], Avg Loss: 3332.27
Epoch [900/1000], Avg Loss: 3160.72
Epoch [1000/1000], Avg Loss: 2867.54

Average MSE on test set (1000 Epochs): 2905.95


### Observation (Exercise 1)

**Result: Massive Improvement**
Increasing the training duration to 1,000 epochs drastically improved the model's performance.

* **Quantitative Comparison:** In the original tutorial run (100 epochs), the Test MSE was approximately **23,308**. With 1,000 epochs, the Test MSE dropped to **2,905.95**. This represents an **87% reduction in error**.
* **Convergence Analysis:** The training logs reveal why this happened. At Epoch 100, the loss was still high (~26,797). It wasn't until around Epoch 300 that the loss stabilized below 5,000. This confirms that the original model was severely "under-trained", it had simply not been given enough iterations to find the optimal weights (to descend the gradient to the bottom of the valley).

In [12]:
# EXERCISE 2: Optimized Architecture

# 1. Define a more complex class
class OptimizedDiabetesModel(nn.Module):
    def __init__(self):
        super(OptimizedDiabetesModel, self).__init__()
        self.layers = nn.Sequential(
            # Layer 1: Expand inputs (10) to 64 neurons (Wider)
            nn.Linear(10, 64),
            nn.ReLU(),

            # Layer 2: 64 -> 32 neurons (Deepening)
            nn.Linear(64, 32),
            nn.ReLU(),

            # Layer 3: 32 -> 16 neurons (Tapering)
            nn.Linear(32, 16),
            nn.ReLU(),

            # Output Layer: 16 -> 1 regression output
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.layers(x)

# 2. Initialize Optimized Model
model_opt = OptimizedDiabetesModel().to(device)
optimizer_opt = optim.Adam(model_opt.parameters(), lr=0.001)

# 3. Train Optimized Model (1000 Epochs)
print("\n Training Optimized Model")
for epoch in range(1000):
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer_opt.zero_grad()
        outputs = model_opt(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer_opt.step()

    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/1000], Loss: {round(loss.item(), 2)}')

# 4. Evaluate Optimized Model
model_opt.eval()
mse_opt = []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model_opt(inputs)
        mse_opt.append(criterion(outputs, targets).item())

avg_mse_opt = np.mean(mse_opt)
print(f"\nOptimized Model Test MSE: {round(avg_mse_opt, 2)}")


 Training Optimized Model
Epoch [200/1000], Loss: 2158.85
Epoch [400/1000], Loss: 5249.96
Epoch [600/1000], Loss: 1346.13
Epoch [800/1000], Loss: 1954.93
Epoch [1000/1000], Loss: 2008.0

Optimized Model Test MSE: 2775.31


### Observation (Exercise 2)

**Result: Marginal Improvement through Architecture**
By expanding the network architecture (making it "wider" with 64 neurons and "deeper" with 3 layers), we achieved a Test MSE of **2,775.31**.

* **Comparison:** This is a slight improvement over the simple model in Exercise 1 (which scored ~2,905). The improvement of roughly **4.5%** indicates that the added complexity allowed the model to capture some additional non-linear patterns in the data that the simpler model missed.
* **Diminishing Returns:** The fact that quadrupling the number of parameters only yielded a small gain suggests we are approaching the **limit of the dataset**. With only ~442 data points, adding even more layers would likely lead to "overfitting" (where the model memorizes the training data but performs worse on the test set) rather than genuine learning.